[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Trusted-AI/AIF360/blob/master/examples/sklearn/demo_mdss_bias_scan.ipynb)

# Identifying Significant Predictive Bias in Classifiers

In this notebook, we attempt to recreate the analysis by Zhe Zhang and Daniel Neill in [Identifying Significant Predictive Bias in Classifiers](https://arxiv.org/pdf/1611.08292.pdf).

The analysis is broken down into three steps, starting with a model trained on COMPAS decile scores only. After running bias scan, we add the distinguishing feature, priors count, to the model. We scan again and train a third model with the new subgroups accounted for. Finally, we reproduce Figure 2 from the paper.

In [1]:
# Install AIF360
!pip install 'aif360'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(context='talk', style='whitegrid')

from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from aif360.sklearn.datasets import fetch_compas
from aif360.sklearn.metrics import mdss_bias_scan, mdss_bias_score

ModuleNotFoundError: No module named 'seaborn'

Data loading

In [ ]:
cols = ['sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree', 'decile_score']
X, y = fetch_compas(usecols=cols)
# Quantize priors count between 0, 1-5, and >5
X['priors_count'] = pd.cut(X['priors_count'], [-1, 0, 5, 100],
                           labels=['0', '1 to 5', 'More than 5'])
X

### 1. Decile score only

In [ ]:
dec = X[['decile_score']]
northpointe = LogisticRegression(penalty='none').fit(dec, y)
y_prob = northpointe.predict_proba(dec)[:, 1]

f, ax = plt.subplots(figsize=(6, 6))
RocCurveDisplay.from_estimator(northpointe, dec, y, ax=ax);

In [ ]:
df = pd.concat([X, pd.Series(1-y_prob, name='recid_prob', index=X.index)], axis=1)
orig_clf = df.groupby('decile_score').mean().recid_prob

#### Privileged group

"Privileged" in this case means the model underestimates the probability of recidivism (overestimates favorable outcomes) for this subgroup. This leads to advantage for those individuals.

In [ ]:
priv_sub, priv_score = mdss_bias_scan(y, y_prob, X=X, pos_label='Survived',
                                      penalty=0.5, privileged=True)
priv = df[priv_sub.keys()].isin(priv_sub).all(axis=1)
priv_sub, priv_score

Note: we show probabilities of recidivism but bias scanning is done with respect to the positive label, 'Survived'.

In [ ]:
print(f'Observed: {y[priv].cat.codes.mean():.2%}')
print(f'Expected: {df[priv].recid_prob.mean():.2%}')
print(f'n = {sum(priv)}')

#### Unprivileged group

"Unprivileged" means the model overestimates the probability of recidivism (underestimates favorable outcomes) for this subgroup. This disadvantages those individuals.

In [ ]:
unpriv_sub, unpriv_score = mdss_bias_scan(y, y_prob, X=X, pos_label='Survived',
                                          penalty=0.5, privileged=False)
unpriv = df[unpriv_sub.keys()].isin(unpriv_sub).all(axis=1)
unpriv_sub, unpriv_score

In [ ]:
print(f'Observed: {y[unpriv].cat.codes.mean():.2%}')
print(f'Expected: {df[unpriv].recid_prob.mean():.2%}')
print(f'n = {sum(unpriv)}')

### 2. Decile score + priors count

In [ ]:
dec = dec.assign(priors_count=X['priors_count'].cat.codes)
northpointe = LogisticRegression(penalty='none').fit(dec, y)
y_prob_pc = northpointe.predict_proba(dec)[:, 1]

In [ ]:
df = pd.concat([X, pd.Series(1-y_prob_pc, name='recid_prob', index=X.index)], axis=1)

#### Privileged group

In [ ]:
priv_sub, priv_score = mdss_bias_scan(y, y_prob_pc, X=X, pos_label='Survived',
                                      penalty=1, privileged=True)
priv = df[priv_sub.keys()].isin(priv_sub).all(axis=1)
priv_sub, priv_score

In [ ]:
print(f'Observed: {y[priv].cat.codes.mean():.2%}')
print(f'Expected: {df[priv].recid_prob.mean():.2%}')
print(f'n = {sum(priv)}')

priv_unpen = mdss_bias_score(y, y_prob_pc, X=X, subset=priv_sub,
                             pos_label='Survived', privileged=True, penalty=0)
print(f'unpenalized score: {priv_unpen:.2f}')

#### Unprivileged group

In [ ]:
unpriv_sub, unpriv_score = mdss_bias_scan(y, y_prob_pc, X=X, pos_label='Survived',
                                          penalty=0.25, privileged=False, n_iter=25)
unpriv = df[unpriv_sub.keys()].isin(unpriv_sub).all(axis=1)
unpriv_sub, unpriv_score

In [ ]:
print(f'Observed: {y[unpriv].cat.codes.mean():.2%}')
print(f'Expected: {df[unpriv].recid_prob.mean():.2%}')
print(f'n = {sum(unpriv)}')

unpriv_unpen = mdss_bias_score(y, y_prob_pc, X=X, subset=unpriv_sub,
                               pos_label='Survived', privileged=False, penalty=0)
print(f'unpenalized score: {unpriv_unpen:.2f}')

### 3. Decile score + priors count + top groups

In [ ]:
df['group'] = 'neither'
df.loc[priv, 'group'] = 'under-estimated'
df.loc[unpriv, 'group'] = 'over-estimated'
df['group'] = df.group.astype('category')
df.head()

In [ ]:
dec = pd.concat([dec, pd.get_dummies(df.group)], axis=1)
northpointe = LogisticRegression(penalty='none').fit(dec, y)
y_prob_pcg = northpointe.predict_proba(dec)[:, 1]
df['recid_prob'] = 1 - y_prob_pcg

In [ ]:
p = sns.relplot(data=df.groupby(['decile_score', 'priors_count', 'group']).mean(),
                x='decile_score', y='recid_prob', hue='priors_count',
                style='priors_count', palette=['r', 'g', 'b'],
                markers=['o', 's', '^'], col='group', s=250)
for ax in p.axes.flatten():
    ax.plot(range(1, 11), orig_clf, '--k')
plt.ylim([0, 1]);
plt.yticks(np.linspace(0, 1., 5));
plt.xticks(range(1, 11));